<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-experiments/haystack_2x_dynamic_prompt_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/deepset-ai/haystack.git@dynamic_prompt_builder#egg=farm-haystack[preview]

DEPRECATION: git+https://github.com/deepset-ai/haystack.git@dynamic_prompt_builder#egg=farm-haystack[preview] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import getpass
import os
import json
import logging

from typing import List

from haystack.preview.components.builders import DynamicPromptBuilder
from haystack.preview.components.generators import GPTGenerator
from haystack.preview.components.generators.chat import GPTChatGenerator
from haystack.preview.dataclasses import ChatMessage, Document
from haystack.preview import component, Pipeline


logging.disable(logging.CRITICAL)

### Enter OpenAI API key

In [3]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")

Enter LLM provider api key:··········


### Use case 1 - ChatMessage(s), no pipeline variables

In [4]:
# no parameter init, we don't use any runtime template variables
prompt_builder = DynamicPromptBuilder()
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo")

In [7]:
pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("prompt_builder.prompt", "llm.messages")

In [8]:
location = "Berlin"
messages = [ChatMessage.from_system("Always respond in German even if some input data is in other languages. Be brief."),
            ChatMessage.from_user("Tell me about {{location}}")]

pipe.run(data={"prompt_builder": {"template_variables":{"location": location}, "prompt_source": messages}})

{'llm': {'replies': [ChatMessage(content='Berlin ist die Hauptstadt Deutschlands und eine der wichtigsten Städte Europas. Sie hat eine reiche Geschichte und ist für ihre lebendige Kultur, ihre multikulturelle Vielfalt und ihre pulsierende Nachtleben bekannt. Berlin bietet eine Vielzahl von Sehenswürdigkeiten und Attraktionen, darunter das Brandenburger Tor, den Berliner Dom, den Checkpoint Charlie und den Alexanderplatz. Die Stadt hat auch eine blühende Kunst- und Musikszene sowie eine Vielzahl von Museen, Galerien und Theatern. Berlin ist außerdem ein wichtiges wirtschaftliches und politisches Zentrum und beheimatet zahlreiche Unternehmen, Institutionen und Botschaften.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, metadata={'model': 'gpt-3.5-turbo-0613', 'index': 0, 'finish_reason': 'stop', 'usage': {'prompt_tokens': 32, 'completion_tokens': 158, 'total_tokens': 190}})]}}

### Use case 2 - ChatMessage(s), pipeline variables
#### We'll use the pipeline that has both runtime inputs (i.e. documents) as well as user provided template variables (i.e. query)

In [9]:
# we'll use documents runtime variables in our template, so we'll let DynamicPromptBuilder know
prompt_builder = DynamicPromptBuilder(expected_runtime_variables=["documents"])
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo")


@component
class DocumentProducer:

  @component.output_types(documents=List[Document])
  def run(self, doc_input: str):
    return {"documents": [Document(content=doc_input)]}


pipe = Pipeline()
pipe.add_component("doc_producer", DocumentProducer())
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("doc_producer.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.messages")

In [10]:
messages = [ChatMessage.from_system("Be helpful assistant, but brief!"),
            ChatMessage.from_user("Here is the document: {{documents[0].content}} Now, answer the following: {{query}}")]

pipe.run(data={"doc_producer": {"doc_input": "Hello world, I'm Haystack!"},
               "prompt_builder": {"prompt_source": messages, "template_variables":{"query": "who's making a greeting?"}}})

{'llm': {'replies': [ChatMessage(content='The greeting is being made by Haystack.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, metadata={'model': 'gpt-3.5-turbo-0613', 'index': 0, 'finish_reason': 'stop', 'usage': {'prompt_tokens': 43, 'completion_tokens': 9, 'total_tokens': 52}})]}}

### Use case 3 - non-chat messages
#### We'll use dynamic templates for non-chat messages

In [12]:
prompt_builder = DynamicPromptBuilder(expected_runtime_variables=["documents"], chat_mode=False)
llm = GPTGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo")


@component
class DocumentProducer:

  @component.output_types(documents=List[Document])
  def run(self, doc_input: str):
    return {"documents": [Document(content=doc_input)]}


pipe = Pipeline()
pipe.add_component("doc_producer", DocumentProducer())
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("doc_producer.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.prompt")

#### Pipeline is setup, but we change template for each pipeline invocation

In [14]:
template = "Here is the document: {{documents[0].content}} Now, answer the following: {{query}}"
pipe.run(data={"doc_producer": {"doc_input": "Hello world, my name is Vladimir"},
               "prompt_builder": {"prompt_source": template, "template_variables":{"query": "who's making a greeting?"}}})

{'llm': {'replies': ['The greeting is being made by the person who wrote the document, which is Vladimir.'],
  'metadata': [{'model': 'gpt-3.5-turbo-0613',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'prompt_tokens': 31,
     'completion_tokens': 17,
     'total_tokens': 48}}]}}

In [15]:
template = "Here is the document: {{documents[0].content}} \n Answer: {{query}}"
pipe.run(data={"doc_producer": {"doc_input": "Hello world, I live in Berlin"},
               "prompt_builder": {"prompt_source": template, "template_variables":{"query": "Where does the speaker live?"}}})

{'llm': {'replies': ['The speaker lives in Berlin.'],
  'metadata': [{'model': 'gpt-3.5-turbo-0613',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'prompt_tokens': 28,
     'completion_tokens': 6,
     'total_tokens': 34}}]}}